In [1]:
import theano.sandbox.cuda
theano.sandbox.cuda.use('gpu6')

Using cuDNN version 5105 on context None
Mapped name None to device cuda: GeForce GTX TITAN X (0000:04:00.0)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 6: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5105)


In [2]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
import cPickle as pickle

In [3]:
def save_network(filename, param_values):
    with open(filename, 'wb') as f:
        pickle.dump(param_values, f, protocol=cPickle.HIGHEST_PROTOCOL)
        
def load_network(filename):
    with open(filename, 'rb') as f:
        param_values = pickle.load(f)
    return param_values

In [4]:
from models.resnet50 import build_model

In [5]:
resnet_data = load_network('models/resnet50.pkl')
means = resnet_data['mean_image']
resnet_params = resnet_data['values']

In [6]:
resnet = build_model()
lasagne.layers.set_all_param_values(resnet.values(), resnet_params)

### Model

In [7]:
input_X = T.tensor4('X', dtype="float32")
target_y = T.vector('y', dtype="int64") 

In [8]:
hidden = lasagne.layers.get_output(resnet['pool5'], input_X, deterministic=True)

In [9]:
hidden_fun = theano.function([input_X], hidden, allow_input_downcast=True)

### Load data

In [10]:
from scipy.misc import imread, imresize

In [11]:
def preprocess(img):
    im = np.swapaxes(np.swapaxes(img, 2, 3), 1, 2)
    im = im[:, ::-1, :, :]
    return im.astype(np.float32)

In [12]:
X_train = np.zeros((25000, 224, 224, 3), dtype=np.float32)

In [13]:
for i in xrange(12500):
    dog = imread('train/dog.%d.jpg' % i)
    cat = imread('train/cat.%d.jpg' % i)
    X_train[i] = imresize(dog, size=(224, 224))
    X_train[i + 12500] = imresize(cat, size=(224, 224))

In [14]:
X_test = np.zeros((1000, 224, 224, 3))
for i in xrange(1000):
    image = imread('test/%d.jpg' % i)
    X_test[i] = imresize(image, size=(224, 224))

In [16]:
X_train = preprocess(X_train)
X_test = preprocess(X_test)

In [17]:
X_train -= means
X_test -= means

### Forward

In [18]:
import tqdm

In [19]:
X_train_resnet = np.zeros((X_train.shape[0], 2048))
batchsize = 256
for start in tqdm.tqdm(xrange(0, X_train.shape[0], batchsize)):
    end = min(start + batchsize, X_train.shape[0])
    X_train_resnet[start:end] = hidden_fun(X_train[start:end]).reshape(-1, 2048)

100%|██████████| 98/98 [05:26<00:00,  3.26s/it]


In [15]:
X_test_resnet = np.zeros((X_test.shape[0], 2048))
for start in tqdm.tqdm(xrange(0, X_test.shape[0], batchsize)):
    end = min(start + batchsize, X_test.shape[0])
    X_test_resnet[start:end] = hidden_fun(X_test[start:end]).reshape(-1, 2048)

100%|██████████| 4/4 [00:09<00:00,  2.45s/it]


In [16]:
np.savez('data_resnet', X_test=X_test_resnet, X_train=X_train_resnet)